In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
# Laste poll-of-polls data
df = pd.read_csv("../data/all_polls.csv", sep=",", encoding='ISO-8859-1')

In [3]:
# Convert to datetime and set the date to the end of the month
df["Dato"] = pd.to_datetime(df["Dato"])

In [4]:
# Sort values and set index
df = df.sort_values("Dato")
df.set_index("Dato", inplace=True)

In [5]:
df_en=df[["Ap","Høyre","Frp","SV","Sp","KrF","Venstre","MDG","Rødt", "Andre"]]

In [6]:
df

,Måling,Ap,Høyre,Frp,SV,Sp,KrF,Venstre,MDG,Rødt,Andre,Year,Month
Dato,,,,,,,,,,,,,
2008-01-09,Verian/TV2,32.7,14.1,23.2,7.1,6.4,7.8,6.0,0.0,0.0,0.0,2008,1
2008-01-10,Sentio/BT,30.7,16.2,23.4,8.1,5.4,5.6,7.4,0.0,0.0,0.0,2008,1
2008-01-18,Norstat/NRK,29.5,17.7,25.6,6.4,5.0,6.4,6.1,0.0,0.0,0.0,2008,1
2008-01-21,InFact/VG,28.4,20.6,24.7,5.6,6.1,6.7,5.2,0.0,1.6,0.0,2008,1
2008-01-21,Respons/Aftenp,30.7,18.3,24.2,7.1,5.4,6.2,5.4,0.0,1.1,0.0,2008,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-05,Verian/TV2,27.7,15.0,21.0,5.8,5.8,4.2,3.9,6.4,5.3,4.8,2025,9
2025-09-05,Norstat/NRK / VL / Dagbl.,26.4,14.3,20.4,6.0,6.2,4.2,4.3,7.2,6.9,4.1,2025,9
2025-09-06,Norfakta/Nat. / KK,27.0,13.5,20.8,5.3,7.0,5.8,4.1,6.0,5.6,4.9,2025,9


In [7]:
df_en

,Ap,Høyre,Frp,SV,Sp,KrF,Venstre,MDG,Rødt,Andre
Dato,,,,,,,,,,
2008-01-09,32.7,14.1,23.2,7.1,6.4,7.8,6.0,0.0,0.0,0.0
2008-01-10,30.7,16.2,23.4,8.1,5.4,5.6,7.4,0.0,0.0,0.0
2008-01-18,29.5,17.7,25.6,6.4,5.0,6.4,6.1,0.0,0.0,0.0
2008-01-21,28.4,20.6,24.7,5.6,6.1,6.7,5.2,0.0,1.6,0.0
2008-01-21,30.7,18.3,24.2,7.1,5.4,6.2,5.4,0.0,1.1,0.0
...,...,...,...,...,...,...,...,...,...,...
2025-09-05,27.7,15.0,21.0,5.8,5.8,4.2,3.9,6.4,5.3,4.8
2025-09-05,26.4,14.3,20.4,6.0,6.2,4.2,4.3,7.2,6.9,4.1
2025-09-06,27.0,13.5,20.8,5.3,7.0,5.8,4.1,6.0,5.6,4.9


In [8]:
# --- 2) Aggregate to quarter-day bins ---
# Resample to 6-hour bins ("6H")
quarter_day = df_en.resample("6H").mean()

# Optional: add how many original obs contributed to each bin
counts = df_en.resample("6H").size().rename("count_per_bin")
quarter_day = quarter_day.join(counts)

# --- 3) Reindex to a complete 6-hour grid ---
all_bins = pd.date_range(quarter_day.index.min(), quarter_day.index.max(), freq="6H")
quarter_day = quarter_day.reindex(all_bins)

# --- 4) Fill missing values with previous day's average ---
# Group by calendar date and take mean (daily average)
daily_avg = quarter_day.resample("1D").mean()

# For each missing bin, fill with previous day's daily average
filled = quarter_day.copy()
for ts, row in filled.iterrows():
    if row.isna().any():  # if any NaN in this bin
        prev_day = (ts - pd.Timedelta(days=1)).normalize()
        if prev_day in daily_avg.index:
            filled.loc[ts] = daily_avg.loc[prev_day]
        else:
            # If no previous day exists, forward-fill from last known
            filled.loc[ts] = filled.loc[:ts].ffill().iloc[-1]

# --- 5) Mark imputed bins (optional, good for model input) ---
was_missing = quarter_day.isna().any(axis=1)
filled["was_imputed"] = was_missing.astype(int)

# --- 6) Re-normalize to ensure sum = 100 ---
cols = [c for c in filled.columns if c not in ["count_per_bin", "was_imputed"]]
sums = filled[cols].sum(axis=1)
filled[cols] = filled[cols].div(sums, axis=0).mul(100)


C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_24076\3856729319.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  quarter_day = df_en.resample("6H").mean()
C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_24076\3856729319.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  counts = df_en.resample("6H").size().rename("count_per_bin")
C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_24076\3856729319.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_bins = pd.date_range(quarter_day.index.min(), quarter_day.index.max(), freq="6H")
C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_24076\3856729319.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype fi

In [9]:
filled

,Ap,Høyre,Frp,SV,Sp,KrF,Venstre,MDG,Rødt,Andre,count_per_bin,was_imputed
2008-01-09 00:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000,1.0,0
2008-01-09 06:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000,0.0,1
2008-01-09 12:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000,0.0,1
2008-01-09 18:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000,0.0,1
2008-01-10 00:00:00,31.714876,16.735537,24.173554,8.367769,5.578512,5.785124,7.644628,0.000000,0.000000,0.000000,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-05 00:00:00,27.063532,14.657329,20.710355,5.902951,6.003002,4.202101,4.102051,6.803402,6.103052,4.452226,2.0,0
2025-09-05 06:00:00,28.014007,13.906953,20.310155,6.903452,5.552776,5.002501,4.202101,6.103052,5.852926,4.152076,0.5,1
2025-09-05 12:00:00,28.014007,13.906953,20.310155,6.903452,5.552776,5.002501,4.202101,6.103052,5.852926,4.152076,0.5,1
2025-09-05 18:00:00,28.014007,13.906953,20.310155,6.903452,5.552776,5.002501,4.202101,6.103052,5.852926,4.152076,0.5,1


In [10]:
df_en=filled[["Ap","Høyre","Frp","SV","Sp","KrF","Venstre","MDG","Rødt", "Andre"]]

In [17]:
#n_timesteps = 5   # past steps to look at
#n_future = 12      # steps into the future we want to predict
n_features = df_en.shape[1]  
series_names = df_en.columns
window_size=10*4

In [12]:
df_en

,Ap,Høyre,Frp,SV,Sp,KrF,Venstre,MDG,Rødt,Andre
2008-01-09 00:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000
2008-01-09 06:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000
2008-01-09 12:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000
2008-01-09 18:00:00,33.607400,14.491264,23.843782,7.297020,6.577595,8.016444,6.166495,0.000000,0.000000,0.000000
2008-01-10 00:00:00,31.714876,16.735537,24.173554,8.367769,5.578512,5.785124,7.644628,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
2025-09-05 00:00:00,27.063532,14.657329,20.710355,5.902951,6.003002,4.202101,4.102051,6.803402,6.103052,4.452226
2025-09-05 06:00:00,28.014007,13.906953,20.310155,6.903452,5.552776,5.002501,4.202101,6.103052,5.852926,4.152076
2025-09-05 12:00:00,28.014007,13.906953,20.310155,6.903452,5.552776,5.002501,4.202101,6.103052,5.852926,4.152076
2025-09-05 18:00:00,28.014007,13.906953,20.310155,6.903452,5.552776,5.002501,4.202101,6.103052,5.852926,4.152076


In [13]:
#n_timesteps = 1  # number of past days to look at
#X, y = [], []

#for i in range(len(df_en) - n_timesteps):
#    X.append(df_en.values[i:i+n_timesteps, :])  
#    y.append(df_en.values[i+n_timesteps, :])  #

#X = np.array(X)  # shape: (num_samples, n_timesteps, 9)
#y = np.array(y)  # shape: (num_samples, 9)

In [14]:
def windowed_dataset_multivariate(series, window_size, batch_size, shuffle_buffer):
    """
    Creates a tf.data.Dataset for multivariate time series.
    
    series: numpy array or tf.Tensor with shape (num_timesteps, num_features)
    window_size: number of timesteps in the input window
    batch_size: training batch size
    shuffle_buffer: buffer size for shuffling
    """
    # Make a Dataset of timesteps
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Create sliding windows of length (window_size + 1)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Convert each window into a batch tensor
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Shuffle windows
    dataset = dataset.shuffle(shuffle_buffer)

    # Split into (input, label): 
    # inputs = first window_size steps, labels = last step
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Batch and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [15]:
#split = int(len(X) * 0.98)
#X_train, X_test = X[:split], X[split:]
#y_train, y_test = y[:split], y[split:]

In [25]:
window_size = 24   # last 6 days (if quarter-day bins)
batch_size = 32
shuffle_buffer = 1000

split = int(len(df_en) * 0.90)
train_data = df_en.values[:split]
val_data   = df_en.values[split:]

train_dataset = windowed_dataset_multivariate(train_data, window_size, batch_size, shuffle_buffer)
val_dataset   = windowed_dataset_multivariate(val_data,   window_size, batch_size, shuffle_buffer=1)


In [26]:
val_data

array([[17.58241758, 25.47452547, 13.08691309, ...,  3.996004  ,
         6.79320679,  6.39360639],
       [17.58241758, 25.47452547, 13.08691309, ...,  3.996004  ,
         6.79320679,  6.39360639],
       [17.58241758, 25.47452547, 13.08691309, ...,  3.996004  ,
         6.79320679,  6.39360639],
       ...,
       [28.014007  , 13.90695348, 20.31015508, ...,  6.10305153,
         5.85292646,  4.15207604],
       [28.014007  , 13.90695348, 20.31015508, ...,  6.10305153,
         5.85292646,  4.15207604],
       [27.91472352, 14.42371752, 20.31978681, ...,  5.99600266,
         5.76282478,  4.23051299]])

In [28]:
series = df_en[cols].values.astype(np.float32)

In [ ]:
val_dataset

In [ ]:
# Collect all batches from tf.data.Dataset into arrays
X_test, y_test = [], []
for X_batch, y_batch in val_dataset:  # or test_dataset if you split
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

print("X_test shape:", X_test.shape)  # (num_test, window_size, 9)
print("y_test shape:", y_test.shape)  # (num_test, 9)


In [ ]:
y_pred = model.predict(X_test)  # shape: (num_test, 9)

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
for i, col in enumerate(df_en.columns):
    plt.subplot(3, 4, i+1)
    plt.plot(y_test[:, i], label='Actual')
    plt.plot(y_pred[:, i], label='Predicted')
    plt.title(col)
    plt.xlabel('Time step')
    plt.ylabel('Value')
    plt.legend()
plt.tight_layout()
plt.show()
